# **DATA CLEANING**

In [206]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import utils.utils as utils

# !pip freeze > ../requirements.txt

In [207]:
data = pd.read_csv(r'..\data\backup\bureau_balance.csv')

## **Data overview**

In [208]:
# print(data.shape)
# # (27299925, 3)

# print(data[:0])
# # Empty DataFrame
# # Columns: [SK_ID_BUREAU, MONTHS_BALANCE, STATUS]
# # Index: []

# data.info()
# # <class 'pandas.core.frame.DataFrame'>
# # RangeIndex: 27299925 entries, 0 to 27299924
# # Data columns (total 3 columns):
# #  #   Column          Dtype 
# # ---  ------          ----- 
# #  0   SK_ID_BUREAU    int64 
# #  1   MONTHS_BALANCE  int64 
# #  2   STATUS          object
# # dtypes: int64(2), object(1)
# # memory usage: 624.8+ MB

In [209]:
utils._calculate_missing_percentage(data)

,Feature,Missing Value Percentage
0,SK_ID_BUREAU,0.0
1,MONTHS_BALANCE,0.0
2,STATUS,0.0


In [210]:
# data.MONTHS_BALANCE.unique()
# # array([  0,  -1,  -2,  -3,  -4,  -5,  -6,  -7,  -8,  -9, -10, -11, -12,
# #        -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25,
# #        -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38,
# #        -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51,
# #        -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, -63, -64,
# #        -65, -66, -67, -68, -69, -70, -71, -72, -73, -74, -75, -76, -77,
# #        -78, -79, -80, -81, -82, -83, -84, -85, -86, -87, -88, -89, -90,
# #        -91, -92, -93, -94, -95, -96])

# data.STATUS.unique()
# # array(['C', '0', 'X', '1', '2', '3', '5', '4'], dtype=object)

**MONTHS_BALANCE**:
- Gonna keep only nearest 60m records (5 years of credit history) *(assumes I only have those)*

**STATUS**:
- C means closed.
- X means status unknown.
- 0 means no DPD.
- 1 means DPD 1-30.
- 2 means DPD 31-60.
- 3 means DPD 61-90.
- 4 means DPD 91-120.
- 5 means DPD 120+ or sold or written off.

## **Tweaking things**

In [ ]:
# Remove obs with >60ms MONTHS_BALANCE cuz I like it that way
data = data[data['MONTHS_BALANCE']>-60]
data['MONTHS_BALANCE'] = data['MONTHS_BALANCE'] + 60

In [213]:
# Remove obs with closed (C) and unknown (X) status in all snapshot months
mask = data.groupby('SK_ID_BUREAU')['STATUS'].transform(lambda x: not all(x.isin(['C', 'X'])))
data = data[mask]

In [214]:
# Humanizer
def recode_status(val):
    if val in ['C']: return 'C'
    if val in ['X']: return 'X'
    if val in ['0']: return 'DPD0'
    if val in ['1']: return 'DPD1-30'
    if val in ['2']: return 'DPD31-60'
    if val in ['3']: return 'DPD61-90'
    if val in ['4']: return 'DPD91-120'
    if val in ['5']: return 'DPD120+'
    return val
data['STATUS'] = data['STATUS'].apply(recode_status)
data['STATUS'] = data['STATUS'].astype('category')

# data.STATUS.unique()
# # ['C', 'DPD0', 'X', 'DPD1-30', 'DPD31-60', 'DPD91-120', 'DPD61-90', 'DPD120+']
# # Categories (8, object): ['C', 'DPD0', 'DPD1-30', 'DPD120+', 'DPD31-60', 'DPD61-90', 'DPD91-120', 'X']

In [215]:
data['STATUS_NEXT'] = (
	data.sort_values(['MONTHS_BALANCE'], ascending=True)
		.groupby('SK_ID_BUREAU')['STATUS']
		.shift(-1)
)
data['STATUS_NEXT'] = data['STATUS_NEXT'].astype('category')

In [216]:
# Some obs show unusual behavior—specifically, accounts that are already closed but still show DPD in the following month.
# This seems illogical, at least to me. Since the number of such cases is small, I drop them entirely, should be save for further observing.
close_mths_1 = data[(data['STATUS'] == 'C') & (~data['STATUS_NEXT'].isin(['C', None]))]['SK_ID_BUREAU']
close_mths_1 = close_mths_1[close_mths_1.duplicated(keep=False)].unique()
# # array([6547545, 6546715, 6388668, 6202448, 5562970, 5764518])
data = data[~data['SK_ID_BUREAU'].isin(close_mths_1)]

## **Data Rollrate**

As with analysis of roll-rate, I filled all closed snapshot obs with the nearest available DPD value.

In [220]:
data_rollrate = data.copy()

In [ ]:
data_rollrate['STATUS'] = data_rollrate['STATUS'].replace(['C', 'X'], np.nan)
data_rollrate = data_rollrate.sort_values(['SK_ID_BUREAU', 'MONTHS_BALANCE'], ascending=True)
data_rollrate['STATUS'] = data_rollrate.groupby('SK_ID_BUREAU')['STATUS'].ffill()

C:\Users\CTV_HUYNQ1\AppData\Local\Temp\ipykernel_7360\1980782985.py:1: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  data_rollrate['STATUS'] = data_rollrate['STATUS'].replace(['C', 'X'], np.nan)


In [ ]:
data_rollrate = data_rollrate.drop(['STATUS_NEXT'], axis=1)
data_rollrate.to_csv(r'..\data\data_rollrate.csv', index=False)

## **Data Vintage**

In [265]:
data_vintage = data.copy().drop(['STATUS_NEXT'], axis=1)
data_vintage.to_csv(r'..\data\data_vintage.csv', index=False)